<a href="https://colab.research.google.com/github/AmirMotefaker/Twitter-Watch/blob/main/Scrape_Twitter_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Why Scrape Twitter?

- [Twitter](https://twitter.com/Twitter) is a major announcement hub where people and companies publish their announcements. This is a great opportunity to use Twitter to follow industry trends. For example, stock market or crypto market targets could be scraped to predict the future price of a stock or crypto.

- Twitter is also a great source of data for sentiment analysis. You can use Twitter to find out what people think about a certain topic or brand. This is useful for market research, product development, and brand awareness.

- So, if we can scrape Twitter data with Python we can have access to this valuable public information for free!

# Setup Wizard

- We'll approach Twitter scraping in three ways:

  - We'll be using [browser automation toolkit Playwright](https://scrapfly.io/blog/web-scraping-with-playwright-and-python/)
    - This is the easiest way to scrape Twitter as we are using real web browser, so all we have to do is navigate to url, wait for page to load and get the results.

  - We'll also take a look at reverse engineering Twitter's hidden API.
This will be a bit harder but these type of scrapers will be much faster than the browser ones. For this we'll be using [httpx](https://pypi.org/project/httpx/).

  - For ScrapFly users we'll also take a look at ScrapFly SDK which makes the above methods even easier.


- We'll be working with both JSON and HTML response data. So, we'll be using [parsel](https://pypi.org/project/parsel/) to parse HTML and [jamespath for JSON](https://scrapfly.io/blog/parse-json-jmespath-python/).

#### All of these libraries are available for free and can be installed via the pip install terminal command:

   $ pip install httpx playwright parsel jmespath scrapfly-sdk



In [1]:
!pip install httpx playwright parsel jmespath asyncio gevent scrapfly-sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Scraping Tweets

- Twitter is a complicated javascript web application that requires javascript to work. So, for tweet scraping we'll be using [Playwright](https://playwright.dev/) browser automation library.

#### Playwright-based Twitter scraper in Python should look something like this:

  1- We'll start a headless Chrome browser

  2- Navigate it to Tweet page URL like https://twitter.com/Scrapfly_dev/status/1577664612908077062

  3- Wait for the page to load

  4- Retrieve page HTML source

  5- Load HTML to "parsel.Selector"

  6- Use CSS selectors and XPath to extract Tweet details and replies

# Python

In [2]:
from parsel import Selector
from playwright.sync_api import sync_playwright
from playwright.sync_api._generated import Page


def parse_tweets(selector: Selector):
    """
    parse tweets from pages containing tweets like:
    - tweet page
    - search page
    - reply page
    - homepage
    returns list of tweets on the page where 1st tweet is the 
    main tweet and the rest are replies
    """
    results = []
    # select all tweets on the page as individual boxes
    # each tweet is stored under <article data-testid="tweet"> box:
    tweets = selector.xpath("//article[@data-testid='tweet']")
    for i, tweet in enumerate(tweets):
        # using data-testid attribute we can get tweet details:
        found = {
            "text": "".join(tweet.xpath(".//*[@data-testid='tweetText']//text()").getall()),
            "username": tweet.xpath(".//*[@data-testid='User-Names']/div[1]//text()").get(),
            "handle": tweet.xpath(".//*[@data-testid='User-Names']/div[2]//text()").get(),
            "datetime": tweet.xpath(".//time/@datetime").get(),
            "verified": bool(tweet.xpath(".//svg[@data-testid='icon-verified']")),
            "url": tweet.xpath(".//time/../@href").get(),
            "image": tweet.xpath(".//*[@data-testid='tweetPhoto']/img/@src").get(),
            "video": tweet.xpath(".//video/@src").get(),
            "video_thumb": tweet.xpath(".//video/@poster").get(),
            "likes": tweet.xpath(".//*[@data-testid='like']//text()").get(),
            "retweets": tweet.xpath(".//*[@data-testid='retweet']//text()").get(),
            "replies": tweet.xpath(".//*[@data-testid='reply']//text()").get(),
            "views": (tweet.xpath(".//*[contains(@aria-label,'Views')]").re("(\d+) Views") or [None])[0],
        }
        # main tweet (not a reply):
        if i == 0:
            found["views"] = tweet.xpath('.//span[contains(text(),"Views")]/../preceding-sibling::div//text()').get()
            found["retweets"] = tweet.xpath('.//a[contains(@href,"retweets")]//text()').get()
            found["quote_tweets"] = tweet.xpath('.//a[contains(@href,"retweets/with_comments")]//text()').get()
            found["likes"] = tweet.xpath('.//a[contains(@href,"likes")]//text()').get()
        results.append({k: v for k, v in found.items() if v is not None})
    return results


def scrape_tweet(url: str, page: Page):
    """
    Scrape tweet and replies from tweet page like:
    https://twitter.com/Scrapfly_dev/status/1587431468141318146
    """
    # go to url
    page.goto(url)
    # wait for content to load
    page.wait_for_selector("//article[@data-testid='tweet']")  
    # retrieve final page HTML:
    html = page.content()
    # parse it for data:
    selector = Selector(html)
    tweets = parse_tweets(selector)
    return tweets


# # example run:
# with sync_playwright() as pw:
#     # start browser and open a new tab:
#     browser = pw.chromium.launch(headless=False)
#     page = browser.new_page(viewport={"width": 1920, "height": 1080})
#     # scrape tweet and replies:
#     tweet_and_replies = scrape_tweet("httpTrutwitter.com/Scrapfly_dev/status/1587431468141318146", page)
#     print(tweet_and_replies)


# example
from playwright.async_api import async_playwright # need to import this first
from gevent import monkey, spawn
import asyncio
import gevent

monkey.patch_all()
loop = asyncio.new_event_loop()


async def f():
    print("start")
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless=True)
    context = await browser.new_context()
    page = await context.new_page()
    await page.goto(f"https://www.google.com")
    print("done")


def greeny():
    while True:  # and not other_exit_condition
        future = asyncio.run_coroutine_threadsafe(f(), loop)
        while not future.done():
            gevent.sleep(1)


greenlet1 = spawn(greeny)
greenlet2 = spawn(greeny)
#loop.run_forever()

It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
